# Collect tweets from Twitter using snscrape and save onto a SQL DB 

use pyodbc to connect to SQL DB and snscrape to collect tweets

In [ ]:
import pyodbc 
import snscrape.modules.twitter as sntwitter 

In [ ]:
# a function to pull tweets from Twitter using snscrape tool
def pull_tweets(my_query, table_name ):
    
    ''' Pull tweets using snscraper tool
        Input is Twitter search query and a table name to insert onto a SQL DB
    '''

    
    for i, tweet in enumerate(
            sntwitter.TwitterSearchScraper(f'{my_query}').get_items()):
        
        # call insert function to save data to SQL table
        insert_tweets(tweet, table_name)

Let's write a function to get a tweet's attributes and save each tweet onto a DB

In [ ]:
# save tweets onto a DB table
def insert_tweets(tweet, table_name):
    '''
    Params:
    tweet: the tweet that is collected and to be inserted
    tale_name: table of DB to insert the given tweet
    '''

    t_text = tweet.content
    t_id = tweet.id
    t_date = tweet.date
    t_username = tweet.username
    t_userid = ''
    t_retweet_count = 0
    t_fav_count = 0
    t_lang = ''
    t_geo = ''
    t_truncated = False
    t_retweeted = False
    t_mentions = ''


    cursor.execute("""
                        INSERT INTO """ + table_name + """
                       (id, cdate, context, User_name, User_id, Retweet_count, Fav, Hashtag_count, Mention, Geo_enabled )
                       VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                            """,
                   (t_id, t_date, t_text, t_username, t_userid,
                    t_retweet_count, t_fav_count, 0, t_mentions, t_geo))

    print('inserted')

In [ ]:
# main function to run the functions above
if __name__ == "__main__":
    
    server_name = ''#your DB server name
    table_name = '' #your table name
    #SQL table has the following field names:
    #id, cdate, context, User_name, User_id, Retweet_count, Fav,
    # Hashtag_count, Mention, Geo_enabled )
                       
    
    conn = pyodbc.connect(
        'DRIVER={};Server={};Database={};trusted_connection=yes; MultipleActiveResultSets=True;as_dict=True;'.format(
        '{ODBC Driver 17 for SQL Server}', server_name, table_name))

    conn.autocommit = True#to save to db automatically without commit
    
    cursor = conn.cursor()

    # Twitter query to search 
    my_query = '@redditstatus or #reddit' 
    
    # pull tweets from Twitter giving the date window
    pull_tweets(my_query, table_name)